In [1]:
# IDCデータで抽出したデータ(Excel)に対し全需/シェア分析のために情報を自動付加する。

# 1. IDC Excelのピボットで分析したいデータを表示
# 使用するフィールド: Brand, Model_name, Product:Laser, Segment1:Office, Segument3:A3MFP Mono, Format, Tone:Mono, 
#                    A4_Mono_Speed:50-80の数値, A4_Color_Speed, Country, REGION, FX_Year:表示したい年, Units
# フィルター: Country, Product:Laser, Format, Tone:Mono, Segument1:Office
# 行: Segument3: A3MFR Mono, REGION:JP, AP, CHN, Brand, Model_Name, A4_Mono_Speed, A4_Color_Speed, FY_Year
# Σ値: 合計/Units

# 2. このスクリプトを起動して実行すると、IDC Excelを自動的に読み込み、情報を付加したうえで日付+全需&シェア動向_D-Mono.xlsxのデータシートに反映してデスクトップに保存される。
# 途中、メーカー名略称、商品名略称をそれぞれメーカー辞書.xlsx, 商品名辞書.xlsxを参照するが、登録がない場合、修正するように指示されるので修正して再実行する。

# 3. 全需&シェア動向.xlsxを開き、Excelピボットを更新する。フィルタボタンが消えてしまっている場合は、ピボットUIからボタンを表示指定して表示する。
# ※ピボットテーブル分析>データソースの変更>データソースの変更>新しいデータの範囲を指定


In [62]:
import pandas as pd
import datetime

thisyear=datetime.date.today ().year
df = pd.read_excel("20241114_IDC_Tracker_FY09Q4-24Q2.xlsx", sheet_name="Pivot", 
                   keep_default_na=False, na_values=[''], 
                   skiprows=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],
                   header=0)
df.tail()

,Segment3,REGION,Brand,Model_Name,A4_Mono_Speed,A4_Color_Speed,FY_Year,集計
1540,NaN,NaN,NaN,NaN,NaN,NaN,FY23,3213
1541,NaN,NaN,NaN,NaN,NaN,NaN,FY24,1471
1542,NaN,NaN,NaN,AltaLink B8255,55.0,0.0,FY24,238
1543,NaN,NaN,NaN,AltaLink B8270,72.0,0.0,FY24,154
1544,総計,NaN,NaN,NaN,NaN,NaN,NaN,515076


In [63]:
# テーブルの整形
# 1. 最終行に"総計"が入ってしまった場合は最終行を削除
if df['Segment3'].iloc[-1]=='総計':df = df.drop(df.index[-1])  

In [64]:
# 2. NaNを埋める
df[['Segment3','REGION','Brand','Model_Name','A4_Mono_Speed','A4_Color_Speed']]=df[['Segment3','REGION','Brand','Model_Name','A4_Mono_Speed','A4_Color_Speed']].ffill()

In [65]:
# 3. Brand(メーカー名)略称列を作成
company_df = pd.read_excel('メーカー辞書.xlsx')
company_dic=dict(zip(company_df['Brand'],company_df['Brand略称']))
  # 'メーカー辞書.xlsx'に登録されていないものがないかをチェック、ある場合は修正用メーカー辞書.xlsxを出力
if False in [brand in company_dic.keys() for brand in list(set(df['Brand']))]:
    print('メーカー辞書.xlsxに未登録のメーカーがあります。')
    for item in list(set(df['Brand'])):
        if item not in company_dic.keys():
            company_df=pd.concat([company_df,pd.DataFrame([{"Brand":item,"Brand略称":"<登録してください>"}])],ignore_index=True)
    company_df.to_excel("メーカー辞書(修正用).xlsx",index=False)
    exit("終了します。メーカー辞書(修正用).xlsxを修正し、ファイル名を変更して再実行してください")
else:
    df['Brand略称']=[company_dic[val] for val in df['Brand']]

In [67]:
# 4. Model(商品)略称列を作成
def check_data_exist_in_df(df, model_name, region):
    import math
    if region in df.loc[df["Model_Name"]==item].values:
        return "●"
    else:
        return ""

def make_additional_df(df,item):
    import pandas as pd
    return pd.DataFrame({
        "JP":check_data_exist_in_df(df,item,"JP"),
        "AP":check_data_exist_in_df(df,item,"AP"),
        "CHN":check_data_exist_in_df(df,item,"CHN"),
        "NA":check_data_exist_in_df(df,item,"NA"),
        "EU":check_data_exist_in_df(df,item,"EU"),
        "ROW":check_data_exist_in_df(df,item,"ROW"),
        "Brand":"<登録してください>",
        "Model_Name":item,
        "Model_Name略称":"<登録してください>",
        "Mono_Speed":df.loc[df["Model_Name"]==item]["A4_Mono_Speed"],
        "Color_Speed":df.loc[df["Model_Name"]==item]["A4_Color_Speed"]
        }).head(1)

product_df = pd.read_excel('商品辞書.xlsx')
product_dic=dict(zip(product_df['Model_Name'],product_df['Model_Name略称']))

  # '商品辞書.xlsx'に登録されていないものがないかをチェック、ある場合は修正用商品辞書.xlsxを出力
if False in [product in product_dic.keys() for product in list(set(df['Model_Name']))]:
    print('商品辞書.xlsxに未登録の商品があります。')
    for item in list(set(df['Model_Name'])):
        if item not in product_dic.keys():
#            product_df=pd.concat([product_df,pd.DataFrame([{"Model_Name":item,"Model_Name略称":"<登録してください>"}])],ignore_index=True)
            product_df=pd.concat([product_df, make_additional_df(df,item)],ignore_index=True)
    product_df.to_excel("商品辞書(修正用).xlsx",index=False)
    exit("終了します。商品辞書(修正用).xlsxを修正し、ファイル名を変更して再実行してください")
else:
#    df['Model_Name略称']=[product_dic[val] for val in df['Model_Name']]
    df = pd.merge(df, product_df, on='Model_Name', how = 'left')

In [68]:
# 5. A4 Mono Speed層列を作成
df['Speed層']=[int(val/10)*10 for val in df['A4_Mono_Speed']]

In [69]:
# 6. Year列を生成
df['Year']=[int('20'+str(val)[-2:]) for val in df['FY_Year']] 

In [70]:
# 7. JP, AP, CHN, NAを抽出
df=pd.concat([df.loc[df['REGION']=='JP'],df.loc[df['REGION']=='AP'],df.loc[df['REGION']=='CHN'],df.loc[df['REGION']=='NA']])

In [71]:
def save_excel(df, output_path, excel_template_dir):
    import pandas as pd
    import openpyxl
    from openpyxl.styles import Font
    from openpyxl.styles import PatternFill
    import datetime
    import os


    excel_output_filename = (
        datetime.datetime.now().strftime("%Y%m%d") + "_全需&シェア動向_D-Mono.xlsx"
    )
    excel_output_path = output_path + "\\" + excel_output_filename
    
    excel_template_name = "全需&シェア動向_Form.xlsx"
    wb = openpyxl.load_workbook(excel_template_name)

    strong_font = Font(name="ＭＳ Ｐ明朝", size=16, bold=True, color="000000ff")
    cell_color = PatternFill(patternType="solid", fgColor="00CCFFFF")

    excelRow=1
    sheet = wb["データ"]
    for i in range(len(df)):
        data = df.loc[i]
        excelRow=excelRow+1
        sheet.cell(row=excelRow, column=1).value = excelRow-1
        sheet.cell(row=excelRow, column=2).value = data["Segment3"]
        sheet.cell(row=excelRow, column=3).value = data["REGION"]
        sheet.cell(row=excelRow, column=4).value = data["Brand"]
        sheet.cell(row=excelRow, column=5).value = data["Brand略称"]
        sheet.cell(row=excelRow, column=6).value = data["Model_Name"]
        sheet.cell(row=excelRow, column=7).value = data["Model_Name略称"]
#        sheet.cell(row=excelRow, column=7).font = strong_font
#        sheet.cell(row=excelRow, column=7).fill = cell_color
        sheet.cell(row=excelRow, column=8).value = data["Speed層"]
        sheet.cell(row=excelRow, column=9).value = data["A4_Mono_Speed"]
        sheet.cell(row=excelRow, column=10).value = data["A4_Color_Speed"]
        sheet.cell(row=excelRow, column=11).value = data["PaperPath"]
        sheet.cell(row=excelRow, column=12).value = data["Recycle"]
        sheet.cell(row=excelRow, column=13).value = data["FY_Year"]
        sheet.cell(row=excelRow, column=14).value = data["Year"]
        sheet.cell(row=excelRow, column=15).value = data["集計"]
    remove_if_file_exist(excel_output_path)
    wb.save(excel_output_path)
    # wb.save(excel_output_path)
    print("全需&シェア動向 Excel Fileが完成しました")
    return excel_output_path

# 同じファイル名のファイルが存在する場合は削除する
def remove_if_file_exist(path):
    import os

    if os.path.exists(path) == True:
        print("古いファイルがあったので削除します({})".format(os.path.basename(path)))
        os.remove(path)
    return

In [73]:
# Excelに保存
df=df.reindex(columns=['Segment3','REGION','Brand','Brand略称','Model_Name','Model_Name略称','Speed層','A4_Mono_Speed','A4_Color_Speed','PaperPath','Recycle','FY_Year','Year','集計'])
# df.to_excel(datetime.date.today().strftime("%Y%m%d")+"D-Mono_Analyze.xlsx")

import os


excel_template_dir = "."
output_path = os.getenv("HOMEDRIVE") + os.getenv("HOMEPATH") + "\\Desktop"
save_excel(df, output_path, excel_template_dir)



全需&シェア動向 Excel Fileが完成しました


'C:\\Users\\fx22228.DC00\\Desktop\\20241119_全需&シェア動向_D-Mono.xlsx'

In [51]:
df

,Segment3,REGION,Brand,Brand略称,Model_Name,Model_Name略称,Speed層,A4_Mono_Speed,A4_Color_Speed,PaperPath,Recycle,FY_Year,Year,集計
0,A3MFP Mono,JP,NaN,Canon,imageRUNNER ADVANCE 6065-R,対象外,60,65.0,0.0,NaN,NaN,FY20,2020,37
1,A3MFP Mono,JP,NaN,Canon,imageRUNNER ADVANCE 6265-R,対象外,60,65.0,0.0,NaN,NaN,FY20,2020,112
2,A3MFP Mono,JP,NaN,Canon,imageRUNNER ADVANCE 6265-R,対象外,60,65.0,0.0,NaN,NaN,FY21,2021,30
3,A3MFP Mono,JP,NaN,Canon,imageRUNNER ADVANCE 6265-R,対象外,60,65.0,0.0,NaN,NaN,FY22,2022,1
4,A3MFP Mono,JP,NaN,Canon,imageRUNNER ADVANCE 6560 III,iRA,60,60.0,0.0,NaN,NaN,FY20,2020,155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,A3MFP Mono,NA,NaN,XC,AltaLink B8170,AltaLink,70,72.0,0.0,NaN,NaN,FY22,2022,4067
1130,A3MFP Mono,NA,NaN,XC,AltaLink B8170,AltaLink,70,72.0,0.0,NaN,NaN,FY23,2023,3213
1131,A3MFP Mono,NA,NaN,XC,AltaLink B8170,AltaLink,70,72.0,0.0,NaN,NaN,FY24,2024,1471
1132,A3MFP Mono,NA,NaN,XC,AltaLink B8255,AltaLink,50,55.0,0.0,NaN,NaN,FY24,2024,238


In [16]:
df2

,Year,REGION,Brand,Year,集計
0,2020,JP,Canon,2020,37
1,2020,JP,Canon,2020,112
2,2021,JP,Canon,2021,30
3,2022,JP,Canon,2022,1
4,2020,JP,Canon,2020,155
...,...,...,...,...,...
1215,2020,NA,Xerox,2020,30
1216,2020,NA,Xerox,2020,46
1217,2021,NA,Xerox,2021,7
1218,2020,NA,Xerox,2020,75
